### API REST para desafio: Peso de animais com grande porte
<br>
<br>

#### Descrição das tecnologias utilizadas:<br>
- Python: por ser uma linguagem de alto nível com boa aceitação no mercado e bem estruturada para sistemas web.<br>
- Flask: biblioteca para criação de sistemas web, leve e bom para ir aumentando o projeto conforme necessidade. <br>
- Flask-RESTFul: biblioteca para criação as API's, com grande abrangência para tal tecnologia. <br>
- Jupyter Lab:editor que venho trabalhando e uso para apresentar o material descritivo conforme necessidade, trabalhando com os notebooks posso gerar documento estruturando texto e código, sendo bom para uma apresentação, gera o arquivo .ipynb.<br>
- Github: https://github.com/Diogo-Esteves/API_peso_posicao_hora <br>
- Link API em "prod": <br>


#### Abaixo estão os arquivos .py e comandos que utilizei para rodar o projeto via notebooks python:

app.py

In [2]:
%%writefile app.py


from flask import Flask
from flask_restful import Api
from entradas import EntradaDados, ListaDados

app = Flask(__name__)
app.secret_key = 'desafio_bosch'
api = Api(app)

# api.add_resource(EntradaDados, '/entrada/<int:peso>/<float:posicaolat>//<float:posicaolon>/<float:horario>')
api.add_resource(EntradaDados, '/entrada/<int:peso>/')
api.add_resource(ListaDados, '/listadedados')


if __name__ == '__main__':
    app.run(port=5000, debug=False)

Overwriting app.py


create_tables.py

In [3]:
%%writefile create_tables.py

import sqlite3

connection = sqlite3.connect('data_wei_pos_tim.db')
cursor = connection.cursor()

create_table = "CREATE TABLE IF NOT EXISTS dados (peso real, posicaolon float, posicaolat float , horario float)"
cursor.execute(create_table)

connection.commit()

connection.close()

Writing create_tables.py


entradas.py

In [4]:
%%writefile entradas.py


import sqlite3
from flask_restful import Resource, reqparse


class EntradaDados(Resource):
    """ Classe para definir o metodo post para a entrada de dados no banco"""

    parser = reqparse.RequestParser()
    # parser.add_argument('peso',
    #                     type=float,
    #                     required=True,
    #                     help="This field cannot be blank")
    parser.add_argument('posicaolat',
                        type=str,
                        required=True,
                        help="This field cannot be blank")
    parser.add_argument('posicaolon',
                        type=str,
                        required=True,
                        help="This field cannot be blank")
    parser.add_argument('hora',
                        type=str,
                        required=True,
                        help="This field cannot be blank")


    def post(self, peso):
        # if ListaDados.find_by_name():
            # return {'message': "An item with the time '{}' already exists.".format(hora)}, 400

        data = EntradaDados.parser.parse_args()
        item = {'peso': peso, 'hora': data['hora'],
               'posicao': data['posicaolat']+ " "+data['posicaolon']}
        
        try:
            self.insert(item)
        except:
            return {'message': 'An error occurred inserting item'}, 500  # Something went wrong
        
        return item, 201

    @classmethod
    def insert(self, item):
        connection = sqlite3.connect('data_wei_pos_tim.db')
        cursor = connection.cursor()

        query = "INSERT INTO dados VALUES (?, ?, ?)"
        cursor.execute(query, (item['peso'],item['hora'],item['posicao'],))

        connection.commit()
        connection.close()


class ListaDados(Resource):
    """ Classe para definir o metodo get para a coleta de dados no banco"""

    def get(self):
        item = self.find_by_name()
        # if item:
        #     return item
        return {'Entradas': item}, 201

    @classmethod
    def find_by_name(self):
        connection = sqlite3.connect('data_wei_pos_tim.db')
        cursor = connection.cursor()

        query = "SELECT * FROM dados"
        result = cursor.execute(query)

        items = []
        for row in result:
            items.append({'peso': row[0], 'hora': row[1], 'posicao': row[2]})

        if result:
            return items

        connection.commit()
        connection.close()


Writing entradas.py
